In [ ]:
%load_ext dotenv
%dotenv
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from datetime import date

from langchain.cache import SQLiteCache
from langchain.chains.llm import LLMChain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chat_models import ChatAnthropic
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.globals import set_llm_cache
from langchain.schema import HumanMessage
from langchain.vectorstores import Chroma
from pyprojroot import here

from redbox.llm.prompts.chat import (
    CONDENSE_QUESTION_PROMPT,
    STUFF_DOCUMENT_PROMPT,
    WITH_SOURCES_PROMPT,
)

vector_store = Chroma(
    embedding_function=SentenceTransformerEmbeddings(),
    persist_directory="../data/db/",
)

test_user_info = {
    "name": "Liam Wilkinson",
    "email": "liam.wilkinson@cabinetoffice.gov.uk",
    "department": "Cabinet Office",
    "role": "Economic Policy",
    "preferred_language": "British English",
}

In [ ]:
x = SQLiteCache(database_path=os.path.join(here(), os.environ["CACHE_LLM_DB"]))

In [ ]:
x.clear()

In [ ]:
set_llm_cache(
    SQLiteCache(database_path=os.path.join(here(), os.environ["CACHE_LLM_DB"]))
)

llm = ChatAnthropic(anthropic_api_key=os.environ["ANTHROPIC_API_KEY"])

In [ ]:
messages = [HumanMessage(content="Tell me a better joke")]

llm(messages)

In [ ]:
llm(messages)

In [ ]:
chat_history = [
    HumanMessage(
        content="You're an expert in geography. Answer geographical questions with authority."
    )
]

user_question = [HumanMessage(content="What is the capital of Spain?")]

docs_with_sources_chain = load_qa_with_sources_chain(
    llm,
    chain_type="stuff",
    prompt=WITH_SOURCES_PROMPT,
    document_prompt=STUFF_DOCUMENT_PROMPT,
    verbose=True,
)

condense_question_chain = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)

# split chain manualy, so that the standalone question doesn't leak into chat
# should we display some waiting message instead?
standalone_question = condense_question_chain(
    {
        "question": user_question,
        "chat_history": chat_history,
        # "user_info": user_info,
        # "current_datetime": datetime.now().isoformat()
    }
)["text"]

docs = vector_store.as_retriever().get_relevant_documents(
    standalone_question,
)

result = docs_with_sources_chain(
    {
        "question": standalone_question,
        "input_documents": docs,
        "user_info": test_user_info,
        "current_date": date.today().isoformat(),
    }
)

In [ ]:
date.today().isoformat()